<a href="https://colab.research.google.com/github/Linaqruf/Scraper/blob/main/Journey_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title ## **Install Niji Scraper**
from google.colab import drive
from IPython.display import clear_output
import os

mount_drive = False # @param {type:'boolean'}
root_dir    = "/content"
drive_dir   = os.path.join(root_dir, "drive", "MyDrive")
repo_dir    = os.path.join(root_dir, "discordia-archivum")
repo_url    = "https://github.com/Linaqruf/discordia-archivum"

def clone_repo(url, dir, branch):
    if not os.path.exists(dir):
       !git clone -b {branch} {url} {dir}

def mount_to_drive(dir):
    if mount_drive:
        if not os.path.exists(dir):
            drive.mount(os.path.dirname(dir))

def install_repository():
    global infinite_image_browser_dir

    infinite_image_browser_url  = f"https://github.com/zanllp/sd-we"+"bui-infinite-image-browsing.git"
    infinite_image_browser_dir  = os.path.join(root_dir, f"infinite-image-browsing")
    infinite_image_browser_deps = os.path.join(infinite_image_browser_dir, "requirements.txt")

    clone_repo(infinite_image_browser_url, infinite_image_browser_dir, "main")

    !pip install -q --upgrade -r {infinite_image_browser_deps}

def install_dependencies():
    requirements = os.path.join(repo_dir, "requirements.txt")

    !apt install aria2
    !pip install python-dotenv
    !pip install --upgrade -r {requirements}

def main():
    mount_to_drive(drive_dir)
    os.chdir(root_dir)
    clone_repo(repo_url, repo_dir, "main")
    os.chdir(repo_dir)
    install_dependencies()
    install_repository()
    clear_output(wait=True)

main()


In [ ]:
#@title ## **Journey Scraper**
import os

image_dir = "/content/image_dir" #@param {type: "string"}
# @markdown Enter your Discord token below.
token = "" #@param {type: "string"}
channel_id = "1022054094476673085" #@param {type: "string"}
# @markdown Which bot do you want to scrape? This code is optimized to only scrape non-grid images from the Journey bot, so don't worry about cropping.
bot = "niji" #@param ["niji", "mid"]
# @markdown Set the range of messages to scrape here. (This does not range the number of messages to download.)
range = 2000 #@param {type: "number"}
# @markdown To specify the `include_word` and `undesired_word`, separate them with commas (e.g., hito_komoru, touhou). By default, it scrapes the newest Niji model.
include_word = "" #@param {type:"string"}
undesired_word = "--style, --niji 4" #@param {type:"string"}
download_attachments = "single"

def scrape(config):
    args = ""
    for k, v in config.items():
        if k.startswith("_"):
            args += f'"{v}" '
        elif isinstance(v, str):
            args += f'--{k}="{v}" '
        elif isinstance(v, bool) and v:
            args += f"--{k} "
        elif isinstance(v, float) and not isinstance(v, bool):
            args += f"--{k}={v} "
        elif isinstance(v, int) and not isinstance(v, bool):
            args += f"--{k}={v} "

    return args

scrape_config = {
    "token": token,
    "channel_id": channel_id,
    "nijijourney": True if bot == "niji" else False,
    "midjourney": True if bot == "mid" else False,
    "range": range if range else None,
    "prompt": include_word,
    "limit": 2147483647,
    "single": True,
    "undesired_word": undesired_word,
    "download_attachments": True,
    "output_folder": image_dir,

}

scrape_args = scrape(scrape_config)

os.chdir(repo_dir)
!python main.py {scrape_args}


In [ ]:
# @title ## **1.4. Image Browser**
import os
import json
import portpicker
from IPython.display import clear_output
from threading import Thread
from google.colab.output import serve_kernel_port_as_iframe, serve_kernel_port_as_window

%store -r

window_height = 550 #@param {type:"slider", min:0, max:1000, step:1}

main_app          = os.path.join(infinite_image_browser_dir, "app.py")
config_file       = os.path.join(infinite_image_browser_dir, "config.json")
port              = portpicker.pick_unused_port()

config = {
    "outdir_txt2img_samples": image_dir,
}

def write_file(filename, config):
    with open(filename, 'w',) as f:
        json.dump(config, f, indent=4)

def run_app():
    !python {main_app} --port={port} --sd_webui_config={config_file} > /dev/null 2>&1

def launch():
    os.chdir(root_dir)

    os.chdir(image_dir)
    write_file(config_file, config)

    thread = Thread(target=run_app)
    thread.start()

    serve_kernel_port_as_iframe(port, width='100%', height=window_height, cache_in_notebook=False)
    clear_output(wait=True)

launch()